In [70]:
import json
import csv
import re
import sys 
import requests
reload(sys)
sys.setdefaultencoding('UTF-8')
import string


#Definitions
commsite = 'http://www.dailymail.co.uk/reader-comments/p/asset/readcomments/'
start = '?max=100&offset='
end ='&order=desc&rcCache=shout'
headers = 'id', 'dateCreated', 'message', 'assetId', 'assetUrl','assetCommentCount', 'userAlias','userLocation','assetHeadline',   'userIdentifier', 'voteRating', 'voteCount', 'hasProfilePicture'
headersReq = {
    'Host': 'www.dailymail.co.uk',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
}


#Precompile RegExs
timeRegex = re.compile(r"(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):.+?Z")
pubRegex = re.compile(r"<meta property =\"article:published_time\" content=\"(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):.+?")


#UI
#URL = raw_input('Enter Full Daily Mail Article URL: ')
#article = re.search('\/article-(\d+)\/',URL).group(1)

#Setup CSV
outfile  = open('DMData.csv', "wb")
writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
writer.writerow(headers+('wordCount','artCreatedDate','CommentPubAfter(Mins)','AverageWordCount'))

#Isolate post ids from data.csv
fb_ids = []
fb_idss= ['1234','1367474646645613', '1368068083252936', '1367411916651886', '1366269283432816']

with open('data.csv','rb') as csvfile:
    reader = csv.DictReader (csvfile)
    for row in reader:
        if row['is_post']=='1':
            match = re.search('._(.+)',row['id'])
            fb_ids.append(match.group(1))

def scrape(sec2,art2):
    #Definitions
    commsite = 'http://www.dailymail.co.uk/reader-comments/p/asset/readcomments/'
    start = '?max=100&offset='
    end ='&order=desc&rcCache=shout'
    headers = 'id', 'dateCreated', 'message', 'assetId', 'assetUrl','assetCommentCount', 'userAlias','userLocation','assetHeadline',   'userIdentifier', 'voteRating', 'voteCount', 'hasProfilePicture'
    headersReq = {
        'Host': 'www.dailymail.co.uk',
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, sdch',
        'Accept-Language': 'en-US,en;q=0.8',
}
#Find Publish date
    URL = 'http://www.dailymail.co.uk/' + sec2 + '/article-' + art2
    req = requests.get(URL,headers=headersReq)
    lines = req.text
    match = re.search(pubRegex,lines)
    if match:
        pubtime=match.groups()
    pubTime = [match.group(3)+'/'+match.group(2)+'/'+match.group(1)+' '+match.group(4)+':'+match.group(5)]

    #First set of comments
    reqURL = commSite + art2
    req = requests.get (reqURL, headers = headersReq)
    html_source = req.text

    numOfComments = re.findall ('{"total":(\d+)',html_source)
    numOfParents = re.findall (',\"parentCommentsCount":(\d+),',html_source)
    nums = int(numOfComments[0])
    nump = int(numOfParents[0])

    #Get rest of comments
    messagesLen=[]
    n=2
    for i in range (0, nums,100): 
        comment = []
        req = requests.get (reqURL + start + str(i) + end, headers = headersReq)
        data = json.loads(req.text)
        for i in range(len(data['payload']['page'])):
            for header in headers:
                if header == 'dateCreated':
                    match = re.search(timeRegex, (data['payload']['page'][i]['dateCreated']))
                    if match:
                        (data['payload']['page'][i]['dateCreated']) = match.group(3)+'/'+match.group(2)+'/'+match.group(1)+' '+match.group(4)+':'+match.group(5)
                comment.append(data['payload']['page'][i].get(header,'N\\A'))
                if header == 'message':
                    words = str(data['payload']['page'][i]['message'])
                    words = words.translate(None, string.punctuation)
                    words = re.split(r'[^0-9A-Za-z]+',words.strip())
                    wordsCount=str(len(words)).split()
                    messagesLen.append(len(words))
                delta = []
            delta = ['=' + '(' + 'B' + str(n) + '-' + 'O' + str(n) + ')' + '*1440'] #MS Excel format
            averageWordCount = ['=average($N$2' + ':' + 'N' + str(nump+1) + ')']
            comment=comment+wordsCount+pubTime+delta + averageWordCount
            writer.writerow(comment)
            comment = []
            n+=1
            
for id in fb_idss:
    try:
        fb = 'https://www.facebook.com/DailyMail/posts/'+ id
        #print 'Downloading fb ' + id
        req = requests.get(fb) #get DM post on fb  
        #print 'Searching daim.ai URL'
        match = re.search ('https:\/\/l\.facebook.com\/l\.php\?u\=http%3A%2F%2Fdailym\.ai\%2F(.+?)\&amp;',req.text) #get DM minisite article number from fb
        #print 'dailm.ai match found'
        article1 = 'http://dailym.ai/' + match.group(1)
        req = requests.get(article1)

        match = re.search ('<meta content=\"http:\/\/www\.dailymail\.co\.uk\/(.+)\/article-(.+?)\/',req.text) #get full article url
        sec2 = match.group(1)
        art2 = match.group(2)

        scrape (sec2,art2)
        print 'Success with ' + fb
    except:
        print 'Error with ' + fb
    
       
outfile.close()
      
req.connection.close()

In [36]:


import csv

fb_ids = []

with open('data.csv','rb') as csvfile:
    reader = csv.DictReader (csvfile)
    for row in reader:
        if row['is_post']=='1':
            match = re.search('._(.+)',row['id'])
            fb_ids.append(match.group(1))
            

    

In [38]:
print len(fb_ids)

281


In [7]:
import requests, re

fb = 'https://www.facebook.com/DailyMail/posts/1367503979976013'

req = requests.get(fb)

In [18]:
match = re.search ('https:\/\/l\.facebook.com\/l\.php\?u\=http%3A%2F%2Fdailym\.ai\%2F(.+?)\&amp;',req.text)

In [21]:
match.group()

u'https://l.facebook.com/l.php?u=http%3A%2F%2Fdailym.ai%2F28QXw5A&amp;'

In [22]:
article = 'http://dailym.ai/' + match.group(1)

req = requests.get(article)

In [49]:
req.text

u'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html><head><!--class fe.views.page_modules.Head-->\n<!--class fe.views.page_modules.Abe-->\n\n     \n     \n     <script type="text/javascript">var ABE={};ABE.startTime=(new Date).getTime(),ABE._rqueue=[],ABE.reveal=function(e){ABE._rqueue.push(e)}</script>\n     \n   \n\n<!--class fe.views.head_modules.Title-->\n<title>Revealed... the 5-step plan to QUITTING sugar for good | Daily Mail Online</title>\n<!--class fe.views.head_modules.MetaGoogleAds-->\n<!-- google_ad_section_start(name=s1) --><meta content="Revealed,5,step,plan,QUITTING,sugar,good,s,not,hard,sounds" name="keywords" /><meta content="Revealed,5,step,plan,QUITTING,sugar,good,s,not,hard,sounds" name="news_keywords" /><meta content="From scrutinising ingredient labels to filling up on protein-rich foods and taking probiotics, a team of nutritionists reveal their tips on how to eradicate sweet treats fro

In [24]:
match = re.search ('name="description" /><meta content="http://www.dailymail.co.uk/news/article-(.+?)/',req.text)

In [25]:
match.group(1)

u'3654400'

In [69]:
print len(fb_idss)

In [54]:
match = re.search ('<meta content=\"http:\/\/www\.dailymail\.co\.uk\/(.+)\/article-(.+?)\/',req.text)

In [57]:
match.group(2)

u'3651458'

In [71]:
fb_ids

['1367031010023310',
 '1367926599933751',
 '1366394693420275',
 '1366510946741983',
 '1366406466752431',
 '1368806123179132',
 '1368048916588186',
 '1368209626572115',
 '1367293756663702',
 '1366365800089831',
 '1367311276661950',
 '1368468479879563',
 '1367474646645613',
 '1368068083252936',
 '1367411916651886',
 '1366269283432816',
 '1367633129963098',
 '1367389093320835',
 '1365928786800199',
 '1368121433247601',
 '1368794786513599',
 '1366661606726917',
 '1367004763359268',
 '1368425216550556',
 '1368307309895680',
 '1366259986767079',
 '1366115266781551',
 '1366032150123196',
 '1366537633405981',
 '1367421656650912',
 '1366137383446006',
 '1368298149896596',
 '1365873853472359',
 '1366526366740441',
 '1368239126569165',
 '1367912706601807',
 '1368479246545153',
 '1367855113274233',
 '1368980353161709',
 '1367831826609895',
 '1367111586681919',
 '1367082896684788',
 '1369063633153381',
 '1367073116685766',
 '1368946883165056',
 '1368326409893770',
 '1367285123331232',
 '13673671633